# Connex REST API health check

Use this notebook to verify connectivity to the Connex REST API `/v1/status` endpoint.

In [ ]:
import json
import logging
import os
import time
from datetime import datetime, timezone
from typing import Any, Dict, Optional

import requests
from dotenv import load_dotenv

# Load local .env file when present so developers can set credentials locally.
load_dotenv(override=False)

logger = logging.getLogger("connex_health")
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

def test_connex_health(
    session: Optional[requests.Session] = None,
    timeout: float = 10.0,
) -> Dict[str, Any]:
    """Perform a health check against the Connex `/v1/status` endpoint.

    Returns latency (ms), status code, and parsed response payload.
    Raises informative errors when configuration is missing or authentication fails.
    """
    rest_url = os.getenv("CONNEX_REST_URL")
    api_key = os.getenv("CONNEX_API_KEY")
    api_secret = os.getenv("CONNEX_API_SECRET")

    missing = [
        name
        for name, value in {
            "CONNEX_REST_URL": rest_url,
            "CONNEX_API_KEY": api_key,
            "CONNEX_API_SECRET": api_secret,
        }.items()
        if not value
    ]
    if missing:
        raise EnvironmentError(f"Missing Connex configuration: {', '.join(missing)}")

    endpoint = rest_url.rstrip("/") + "/v1/status"

    request_headers = {
        "Accept": "application/json",
        "User-Agent": "connex-health-check/0.1",
        "X-CONNEX-API-KEY": api_key,
        "X-CONNEX-API-SECRET": api_secret,
    }
    sanitized_headers = {
        "X-CONNEX-API-KEY": (
            api_key if not api_key else f"{api_key[:4]}...{api_key[-4:]}"
        ),
        "X-CONNEX-API-SECRET": (
            "***" if not api_secret else f"***{api_secret[-4:]}"
        ),
    }

    http = session or requests.Session()

    logger.info("Querying %s", endpoint)
    logger.info("Using authentication headers: %s", sanitized_headers)

    start = time.perf_counter()
    response = http.get(endpoint, headers=request_headers, timeout=timeout)
    latency_ms = (time.perf_counter() - start) * 1000.0

    try:
        payload = response.json()
        payload_type = "json"
    except ValueError:
        payload = response.text
        payload_type = "text"

    logger.info("Received status %s in %.2f ms", response.status_code, latency_ms)

    if response.status_code == 401:
        raise PermissionError("Connex REST API rejected the provided credentials (401 Unauthorized).")
    response.raise_for_status()

    return {
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "url": endpoint,
        "latency_ms": round(latency_ms, 2),
        "status_code": response.status_code,
        "payload_type": payload_type,
        "payload": payload,
        "sanitized_auth_headers": sanitized_headers,
    }


__all__ = ["test_connex_health"]


In [ ]:
try:
    result = test_connex_health()
    print(json.dumps(result, indent=2, default=str))
except Exception as exc:
    logger.error("Connex health check failed: %s", exc)
    raise
